In [ ]:
# default_exp hpopt
# default_cls_lvl 3

# Hyperparameter Optimization Module
> Pytorch Models for Sequential Data

In [ ]:
#export
from seqdata.core import *
from seqdata.models.core import *
from seqdata.learner import *
from fastai.basics import *
from fastai.callback.schedule import *
from fastai.callback.rnn import *
from fastai.callback.tracker import *

import ray
from ray import tune
from ray.tune import Trainable
from ray.tune.schedulers import *
from ray.tune.trial import ExportFormat

In [ ]:
f_paths = '/mnt/data/Systemidentification/WienerHammerstein/'
hdf_files = L([f for f in get_hdf_files(f_paths) if 'test' not in str(f)])
tfm_src = CreateDict([DfHDFCreateWindows(win_sz=400,stp_sz=100,clm='u')])
dls = DataBlock(blocks=(SequenceBlock.from_hdf(['u'],TensorSequencesInput),
                        SequenceBlock.from_hdf(['y'],TensorSequencesOutput)),
                get_items=tfm_src,
                splitter=ApplyToDict(FuncSplitter(lambda o: 'valid' in str(o)))).dataloaders(hdf_files)

## optimizer core

First we need a log uniform distibution for variables with vast value ranges

In [ ]:
#export
def log_uniform(min_bound, max_bound, base=10):
    '''uniform sampling in an exponential range'''
    logmin = np.log(min_bound) / np.log(base)
    logmax = np.log(max_bound) / np.log(base)
    def _sample():
        return base**(np.random.uniform(logmin, logmax))
    return _sample

In [ ]:
[log_uniform(1e-8, 1e-2)() for _ in range(5)]

[2.209163639289779e-08,
 0.0015573309086515188,
 1.2760465923712972e-07,
 0.001048531946420762,
 4.6015122513630064e-08]

In [ ]:
#export
class LearnerTrainable(tune.Trainable):

    def setup(self, config):
        self.create_lrn = ray.get(config['create_lrn'])
        self.dls = ray.get(config['dls'])

        self.lrn = self.create_lrn(self.dls,config)

    def step(self):
        with self.lrn.no_bar(): self.lrn.fit(1)
        train_loss,valid_loss,rmse = self.lrn.recorder.values[-1]
        result = {'train_loss': train_loss,
                'valid_loss': valid_loss,
                'mean_loss': rmse}
        return result

    def save_checkpoint(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "model.pth")
        torch.save(self.lrn.model.state_dict(), checkpoint_path)
        return tmp_checkpoint_dir

    def load_checkpoint(self, tmp_checkpoint_dir):
        self.lrn.model.load_state_dict(torch.load(tmp_checkpoint_dir))

    def _export_model(self, export_formats, export_dir):
        if export_formats == [ExportFormat.MODEL]:
            path = os.path.join(export_dir, "exported_model")
            torch.save(self.lrn.model.state_dict(), path)
            return {ExportFormat.MODEL: path}
        else:
            raise ValueError("unexpected formats: " + str(export_formats))

    # the learner class will be recreated with every perturbation, saving the model
    # that way the new hyperparameter will be applied
    def reset_config(self, new_config):
        model_state = self.lrn.model.state_dict()
#         import pdb; pdb.set_trace()
        self.lrn = self.create_lrn(self.dls,new_config)
        
        #restore trainable parameters, keeping the new hyperparameters in the model like dropout
        self.lrn.model.load_state_dict(model_state)
        
        self.config = new_config
        return True

NameError: name 'tune' is not defined

In [ ]:
#export
from fastai.callback.tracker import SaveModelCallback 
class CBRaySaveModel(SaveModelCallback):
    "A `TrackerCallback` that saves the model's best during training in a tune checkpoint directory"
    
    def _save(self, name):
        if not hasattr(self,'idx_checkpoint'): self.idx_checkpoint = 0
        with tune.checkpoint_dir(step=self.idx_checkpoint) as checkpoint_dir:
            file = os.path.join(checkpoint_dir,name+'.pth')
            print(file)
            save_model(file, self.learn.model,opt=None)
            self.last_saved_path = file
            self.idx_checkpoint += 1
            
    #do not try to restore model
    def after_fit(self, **kwargs): pass

In [ ]:
#export
def learner_optimize(config, checkpoint_dir=None):
        create_lrn = ray.get(config['create_lrn'])
        dls = ray.get(config['dls'])
        
        #Scheduling Parameters for training the Model
        lrn_kwargs = {'n_epoch':100,'pct_start':0.5}
        for attr in ['n_epoch','pct_start']:
            if attr in config: lrn_kwargs[attr] = config[attr]

        lrn = create_lrn(dls,config)
        
        # load checkpoint data if provided
        if checkpoint_dir:
            lrn.model.load_state_dict(torch.load(tmp_checkpoint_dir))
        
        lrn.lr = config['lr'] if 'lr' in config else 3e-3
        lrn.add_cb(CBRayReporter() if 'reporter' not in config else ray.get(config['reporter'])())
        lrn.add_cb(CBRaySaveModel())
        with lrn.no_bar(): 
            ray.get(config['fit_method'])(lrn,**lrn_kwargs)

The mutation config dictionary consists of functions that sample from a distribution. In order to retrieve a dictionary with one realisation we need the function sample_config

In [ ]:
#export
def sample_config(config):
    ret_conf = config.copy()
    for k in ret_conf:
        ret_conf[k]=ret_conf[k]()
    return ret_conf

In [ ]:
#export
class CBRayReporter(Callback):
    "`Callback` reports progress after every epoch to the ray tune logger"

    def after_epoch(self):
        train_loss,valid_loss,rmse = self.learn.recorder.values[-1]
        tune.report(train_loss=train_loss,
                        valid_loss=valid_loss,
                        mean_loss=rmse)


In [ ]:
#export
class HPOptimizer():
    def __init__(self,create_lrn,dls):
        self.create_lrn = create_lrn
        self.dls = dls
        self.analysis = None
    
    @delegates(ray.init)
    def start_ray(self,**kwargs):
        ray.shutdown()
        ray.init(**kwargs)
        
    def stop_ray(self):
        ray.shutdown()
        
    
        

    @delegates(tune.run, keep=True)
    def optimize(self,config,optimize_func=learner_optimize,resources_per_trial={"gpu": 1.0},verbose=1,**kwargs):
        config['create_lrn'] = ray.put(self.create_lrn)
        #dls are large objects, letting ray handle the copying process makes it much faster
        config['dls'] = ray.put(self.dls) 
        if 'fit_method' not in config: config['fit_method'] = ray.put(Learner.fit_flat_cos)

        self.analysis = tune.run(
            optimize_func,
            config=config,
            resources_per_trial=resources_per_trial,
            verbose=verbose,
            **kwargs)
        return self.analysis
        
    @delegates(tune.run, keep=True)
    def optimize_pbt(self,opt_name,num_samples,config,mut_conf,perturbation_interval=2,
                 stop={"training_iteration": 40 },
                 resources_per_trial={"gpu": 1 },
                 resample_probability=0.25,
                 quantile_fraction=0.25,
                 **kwargs):
        self.mut_conf = mut_conf
        
        config['create_lrn'] = ray.put(self.create_lrn)
        #dls are large objects, letting ray handle the copying process makes it much faster
        config['dls'] = ray.put(self.dls) 
        
        

        
        scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="mean_loss",
        mode="min",
        perturbation_interval=perturbation_interval,
        resample_probability=resample_probability,
        quantile_fraction=quantile_fraction,
        hyperparam_mutations=mut_conf)
        
        self.analysis = tune.run(
            LearnerTrainable,
            name=opt_name,
            scheduler=scheduler,
            reuse_actors=True,
            verbose=1,
            stop=stop,
            checkpoint_score_attr="mean_loss",
            num_samples=num_samples,
            resources_per_trial=resources_per_trial,
            config=config,
            **kwargs)
        return self.analysis
    
    def best_model(self):
        if self.analysis is None: raise Exception
        model = self.create_lrn(self.dls,sample_config(self.mut_conf)).model
        f_path = self.analysis.get_best_trial('mean_loss',mode='min').checkpoint.value
        model.load_state_dict(torch.load(f_path))
        return model

### Test Population Based Training

In [ ]:
def create_lrn(dls,config):
    lr = config['lr']
    alpha = config['alpha']
    beta = config['beta']
    weight_p = config['weight_p']
    
    lrn = RNNLearner(dls,cbs=[TimeSeriesRegularizer(alpha,beta)],weight_p=weight_p)
    lrn.lr = lr
    return lrn

In [ ]:
config={
            "lr": tune.loguniform(1e-2, 1e-4),
            "alpha": tune.loguniform(1e-5, 10),
            "beta": tune.loguniform(1e-5, 10),
            "weight_p": tune.uniform(0, 0.5)}
mut_conf = {# distribution for resampling
            "lr": log_uniform(1e-8, 1e-2),
            "alpha": log_uniform(1e-5, 10),
            "beta": log_uniform(1e-5, 10),
            "weight_p": lambda: np.random.uniform(0, 0.5)}

hp_opt = HPOptimizer(create_lrn,dls)
hp_opt.start_ray()
hp_opt.optimize_pbt('pbt_test',4,config,mut_conf,perturbation_interval=1,
                 stop={"training_iteration": 3 },
                 resources_per_trial={"gpu": 0.5},
                 local_dir='/mnt/data/ray_results')#no cpu count is necessary

Trial name,status,loc,alpha,beta,lr,weight_p,loss,iter,total time (s),train_loss,valid_loss,neg_mean_loss
LearnerTrainable_16e2e_00000,TERMINATED,,0.00107201,0.000472496,0.00313063,0.212251,0.244638,3,7.28212,0.0548574,0.0598967,-0.244638
LearnerTrainable_16e2e_00001,TERMINATED,,0.00504348,7.86733,0.00115469,0.104033,0.245494,3,9.02798,0.0554254,0.0603163,-0.245494
LearnerTrainable_16e2e_00002,TERMINATED,,0.526945,0.0161059,0.0001224,0.203164,0.246905,3,7.46955,0.0562173,0.06102,-0.246905
LearnerTrainable_16e2e_00003,TERMINATED,,0.00134001,8.31061e-05,0.00391329,0.265313,0.241195,3,7.34682,0.0543043,0.0582198,-0.241195


(pid=35087) [0, 0.05542542040348053, 0.06031626835465431, 0.24549369513988495, '00:02']


In [ ]:
hp_opt.best_model()

SimpleRNN(
  (rnn): RNN(
    (rnns): ModuleList(
      (0): GRU(1, 100, batch_first=True)
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
    )
  )
  (final): SeqLinear(
    (lin): Sequential(
      (0): Sequential(
        (0): Conv1d(100, 100, kernel_size=(1,), stride=(1,))
        (1): Mish()
      )
      (1): Conv1d(100, 1, kernel_size=(1,), stride=(1,))
    )
  )
)

### Test Grid Search

In [ ]:
def create_lrn(dls,config):
    lrn = RNNLearner(dls,hidden_size=config['hidden_size'],metrics=[fun_rmse,mse])
    return lrn

In [ ]:
class CustomReporter(Callback):
    "`Callback` reports progress after every epoch to the ray tune logger"

    def after_epoch(self):
        train_loss,valid_loss,rmse,mse = self.learn.recorder.values[-1]
        tune.report(train_loss=train_loss,
                        valid_loss=valid_loss,
                        mean_loss=rmse,
                        mse=mse)

In [ ]:
hp_opt = HPOptimizer(create_lrn,dls)

In [ ]:
hp_opt.start_ray(local_mode=False)

2020-10-28 10:02:21,737	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


In [ ]:
search_space = {
    "hidden_size": tune.grid_search([10,20,50,100]),
    'n_epoch':10,
    'reporter':ray.put(CustomReporter)
}

In [ ]:
hp_opt.optimize(resources_per_trial={"gpu": 0.5},
                config=search_space)

Trial name,status,loc,hidden_size,loss,iter,total time (s),train_loss,valid_loss,mse
learner_optimize_4ac35_00000,TERMINATED,,10,0.245868,10,32.7623,0.0571349,0.0605004,0.0605004
learner_optimize_4ac35_00001,TERMINATED,,20,0.244005,10,30.6205,0.0590238,0.059579,0.059579
learner_optimize_4ac35_00002,TERMINATED,,50,0.217499,10,31.463,0.0512671,0.0473269,0.0473269
learner_optimize_4ac35_00003,TERMINATED,,100,0.0727086,10,30.7753,0.0160919,0.0052895,0.0052895


(pid=26757) Better model found at epoch 9 with valid_loss value: 0.060500435531139374.
(pid=26757) /home/daniel/ray_results/learner_optimize/learner_optimize_4ac35_00000_0_hidden_size=10_2020-10-28_10-02-23/checkpoint_9/model.pth


In [ ]:
hp_opt.analysis.get_best_config('mean_loss',mode='min')

### Test Random Search

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_models.ipynb.
Converted 01a_IndRNN.ipynb.
Converted 02_learner.ipynb.
Converted 03_dataloaders.ipynb.
Converted 11_dualrnn.ipynb.
Converted 12_TensorQuaternions.ipynb.
Converted 13_HPOpt.ipynb.
Converted index.ipynb.
